
Authors: Simon Vandenhende
Licensed under the CC BY-NC 4.0 license (https://creativecommons.org/licenses/by-nc/4.0/)

In [ ]:
!pip install easydict

!pip install imageio
!pip install termcolor

In [ ]:
import argparse

In [ ]:
import PIL.Image
import cv2
import os
import numpy as np
import sys
import torch
import torch_tensorrt
import torchvision.transforms
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from utils.config import create_config
from utils.common_config import get_train_dataset, get_transformations,\
                                get_val_dataset, get_train_dataloader, get_val_dataloader,\
                                get_optimizer, get_model, adjust_learning_rate,\
                                get_criterion
from utils.logger import Logger
from train.train_utils import train_vanilla
from evaluation.evaluate_utils import eval_model, validate_results, save_model_predictions,\
                                    eval_all_results
from termcolor import colored
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
config_env = './configs/env.yml'
config_exp1 = './configs/nyud/hrnet18/mti_net+edges_normals.yml'
config_exp2 = 'C:/Users/ataka/PycharmProjects/CMP712/configs/pascal/hrnet18/mti_net.yml'
image_path = "/home/atakan/Thesis/TensorRT/test_data/"
print("import files handled")

In [ ]:
def eval_my_model(p, test_data, model):
    tasks = p.TASKS.NAMES
    print(colored("Tasks are:", 'blue'), colored(tasks, 'red'))
    #performance_meter = PerformanceMeter(p)
    print("Burada performance meter sadece bilgilendirme amaçlı tutuluyor. Inference time incelenecek.")

    model.eval() #activate eval mode to test.
    images = test_data # Test data must be an image in the form of 
                       # img[1, 3 , 480, 640] # [N, C, H, W]
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    with torch.no_grad():
        start.record()
        output = model(images)
        end.record()

        torch.cuda.synchronize()
        print(start.elapsed_time(end))
    return output

def load_test_data(image_name):
    image = PIL.Image.open(image_path + image_name)
    print (image_name," görüntüsünün boyutları: ", image.size)
    print ("Test için boyutları ayarlanıyor. ") # NYUD icin H = 480, W = 640
    size = (640, 480) # W = 640, H = 480
    image = image.resize(size)
    print ("Yüklenen goruntunun yeni boyutları: ", image.size)
    return image

def convert_data(image):
    transform = transforms.ToTensor()

    tensor_image = transform(image)
    return tensor_image

In [ ]:
torch.autograd.set_detect_anomaly(mode=True)
cv2.setNumThreads(0)
#p = create_config(args.config_env, args.config_exp) orjinal hali
p = create_config(config_env, config_exp1)
sys.stdout = Logger(os.path.join(p['output_dir'], 'log_file.txt'))
print(colored(p, 'red'))
print(torch.cuda.is_available())
print(p)

In [ ]:


# Get model
print(colored('Retrieve model', 'blue'))
model = get_model(p)
model = torch.nn.DataParallel(model)
model = model.cuda()
print(model)
model1 = model
if model1 == model:
    print("Yes they are equal ! Keep going bro!")
else:
    print("You failed f*cking idiot")
#Now load pre trained weights
model.load_state_dict(torch.load(p['checkpoint'])['model'])

print("p[train_db_name] :", p['train_db_name']);
print("trBatch size is: ", p['trBatch'])
print("valBatch size is", p['valBatch'])

test_image = load_test_data("yatak_odasi.jpg")
test_tensor_image = convert_data(test_image)
test_tensor_image = test_tensor_image.unsqueeze(0)
print("Type of test_tensor_image:", type(test_tensor_image))
print("Shape of test_tensor_image:", test_tensor_image.size())

out = eval_my_model(p, test_tensor_image, model)
#print(out)
print("Type of out image: ", type(out))
#print("Shape of out image:", out.size())
for key, value in out.items():
    print (key)

#print(out['semseg'])
print("Type of out[semseg] is: ", type(out['semseg']))
print("Shape of out[semseg] is:", out['semseg'].size())
im = out['semseg']

In [ ]:
print(p)

In [ ]:
def main():
    # Retrieve config file
    # initialize a model with the same architecture as the model which parameters you saved into the .pt/h file


    # load the parameters into the model



    #im = PIL.Image.open("C:/Users/ataka/PycharmProjects/CMP712/databases/PASCAL_MT/semseg/pascal-context/2008_000016.png")
    #im.show()
    #image = np.array(im)
    #from matplotlib import pyplot as plt
    #plt.imshow(image, interpolation='nearest')
    #plt.show()
    #print(np.unique(image))
    torch.autograd.set_detect_anomaly(mode=True)
    cv2.setNumThreads(0)
    #p = create_config(args.config_env, args.config_exp) orjinal hali
    p = create_config(config_env, config_exp1)
    sys.stdout = Logger(os.path.join(p['output_dir'], 'log_file.txt'))
    print(colored(p, 'red'))
    print(torch.cuda.is_available())

    # Get model
    print(colored('Retrieve model', 'blue'))
    model = get_model(p)
    model = torch.nn.DataParallel(model)
    model = model.cuda()
    print(model)
    model1 = model
    if model1 == model:
        print("Yes they are equal ! Keep going bro!")
    else:
        print("You failed f*cking idiot")
    #Now load pre trained weights
    model.load_state_dict(torch.load(p['checkpoint'])['model'])

    print("p[train_db_name] :", p['train_db_name']);
    print("trBatch size is: ", p['trBatch'])
    print("valBatch size is", p['valBatch'])

    test_image = load_test_data("yatak_odasi.jpg")
    test_tensor_image = convert_data(test_image)
    test_tensor_image = test_tensor_image.unsqueeze(0)
    print("Type of test_tensor_image:", type(test_tensor_image))
    print("Shape of test_tensor_image:", test_tensor_image.size())

    out = eval_my_model(p, test_tensor_image, model)
    #print(out)
    print("Type of out image: ", type(out))
    #print("Shape of out image:", out.size())
    for key, value in out.items():
        print (key)

    #print(out['semseg'])
    print("Type of out[semseg] is: ", type(out['semseg']))
    print("Shape of out[semseg] is:", out['semseg'].size())
    im = out['semseg']
    #plt.imshow(im.permute(2, 3, 1))

In [ ]:
"""


    # Get criterion
    print(colored('Get loss', 'blue'))
    criterion = get_criterion(p)
    criterion.cuda()
    print(criterion)

    # CUDNN
    print(colored('Set CuDNN benchmark', 'blue')) 
    torch.backends.cudnn.benchmark = True

    # Optimizer
    print(colored('Retrieve optimizer', 'blue'))
    optimizer = get_optimizer(p, model)
    print(optimizer)

    # Dataset
    print(colored('Retrieve dataset', 'blue'))
    # Transforms
    train_transforms, val_transforms = get_transformations(p)
    train_dataset = get_train_dataset(p, train_transforms)
    print(train_dataset)
    from PIL import Image
    im = Image.open(train_dataset.images[4])
    im.show()
    _im = np.array(im).astype(np.float32)
    print(_im)
    print("Type and shape of image:", type(_im), _im.shape)
    val_dataset = get_val_dataset(p, val_transforms)
    true_val_dataset = get_val_dataset(p, None) # True validation dataset without reshape
    train_dataloader = get_train_dataloader(p, train_dataset)

    val_dataloader = get_val_dataloader(p, val_dataset)
    print('Train samples %d - Val samples %d' %(len(train_dataset), len(val_dataset)))
    print('Train transformations:')
    print(train_transforms)
    print('Val transformations:')
    print(val_transforms)

    # Resume from checkpoint
    if os.path.exists(p['checkpoint']):
        print(colored('Restart from checkpoint {}'.format(p['checkpoint']), 'blue'))
        checkpoint = torch.load(p['checkpoint'], map_location='cpu')
        optimizer.load_state_dict(checkpoint['optimizer'])
        model.load_state_dict(checkpoint['model'])
        start_epoch = checkpoint['epoch']
        best_result = checkpoint['best_result']
        print ("############################################")
        for i in range(0,5):
            print("I AM NOT RETRAINED !!!!")
        print ("############################################")

    else:
        print(colored("########### I am heree!!!!!!!!!!"), 'red')
        print(colored('No checkpoint file at {}'.format(p['checkpoint']), 'blue'))
        start_epoch = 0
        save_model_predictions(p, val_dataloader, model)
        best_result = eval_all_results(p)


    #start_epoch = 0 #Eğer kaydedilen modelin son epochdan sonra devam etmesi isteniyorsa bunun
                    # yorum olarak kalması gerekir.
    # Main loop
    print(colored('Starting main loop', 'blue'))

    for epoch in range(start_epoch, p['epochs']):
        print(colored('Epoch %d/%d' %(epoch+1, p['epochs']), 'yellow'))
        print(colored('-'*10, 'yellow'))

        # Adjust lr
        lr = adjust_learning_rate(p, optimizer, epoch)
        print('Adjusted learning rate to {:.5f}'.format(lr))

        # Train 
        print('Train ...')
        eval_train = train_vanilla(p, train_dataloader, model, criterion, optimizer, epoch)

        # Evaluate
            # Check if need to perform eval first
        if 'eval_final_10_epochs_only' in p.keys() and p['eval_final_10_epochs_only']: # To speed up -> Avoid eval every epoch, and only test during final 10 epochs.
            if epoch + 1 > p['epochs'] - 10:
                eval_bool = True
            else:
                eval_bool = False
        else:
            eval_bool = True

        # Perform evaluation
        if eval_bool:
            print('Evaluate ...')
            save_model_predictions(p, val_dataloader, model)
            curr_result = eval_all_results(p)
            improves, best_result = validate_results(p, curr_result, best_result)
            if improves:
                print('Save new best model')
                torch.save(model.state_dict(), p['best_model'])

        # Checkpoint
        print('Checkpoint ...')
        torch.save({'optimizer': optimizer.state_dict(), 'model': model.state_dict(), 
                    'epoch': epoch + 1, 'best_result': best_result}, p['checkpoint'])

    # Evaluate best model at the end
    print(colored('Evaluating best model at the end', 'blue'))
    model.load_state_dict(torch.load(p['checkpoint'])['model']) #this must be p['best_result']
    save_model_predictions(p, val_dataloader, model)
    eval_stats = eval_all_results(p)
"""
if __name__ == "__main__":
    main()